In [2]:
import pandas

In [3]:
filename = "/Users/simondi/PHD/data/data/target_infect_x/screening_data_subset/cells_sample_10_normalized.tsv"

In [4]:
data = pandas.read_csv(filename, sep="\t", header=0)

In [5]:
feature_cols = [(i, x) for i, x in enumerate(data.columns.values) if x.startswith("cells")]
for i, c in feature_cols:
    data.loc[:, c] = data.loc[:, c].astype('float64')

In [8]:
grops = ['pathogen', 'library', 'design', 'replicate', 'gene']

In [24]:
grop_cnt = data.query("design=='p' and library=='d' and replicate==1 and study=='infectx'").groupby("gene").size()

In [29]:
data.query("design=='p' and library=='d' and replicate==1 and study=='infectx'").groupby("gene").filter(lambda x: len(x) >= 40).groupby("gene").size()

gene
abcc1          40
abcc2          40
acad8          40
acsl4          40
adh1c          40
agxt2          40
aipl1          40
akap7          40
aldh1l2        40
aldh3b1        40
aldh4a1        40
allc           40
alx3           40
ankh           40
apex2          40
aqp3           40
aqpep          60
arpc3        6890
arsi           40
atl1           40
atn1           40
atp13a1        40
atp1b3         40
atp5g3         40
atp6v0d2       40
atp6v1a      6890
atp8b2         40
b3gntl1        40
b4galt2        40
bcam           40
             ... 
thrb           40
thumpd2        40
tkt            40
tln1         4568
tlr4           40
tnfrsf13b      40
tnfrsf9        40
tp63           40
tp73           40
trap1          40
tsc2           40
tsen2          40
ttl            40
tub            40
tubal3         40
tube1          40
uap1           40
ubash3b        40
ugdh           40
ugt3a1         40
unkl           60
unknown      6630
vat1           40
vat1l          40
vav3 

In [56]:
data_new = data.query("library=='d' and design=='p' and replicate==1 and (pathogen=='brucella' or pathogen=='listeria' or pathogen=='adeno' or pathogen=='bartonella')").groupby(["gene"]).filter(lambda x: len(x) == 40)

In [ ]:
data-new